https://www.kaggle.com/c/rossmann-store-sales

#### ‍💼💰📈 **Questão de Negócio:** Previsão de valor de vendas em cada loja nos próximos 6 meses.

**Causa Raiz:** CFO deseja determinar o investimento para reformas de cada loja. Para isso, deseja obter informações futuras.  
**Formato da Solução:**    
    - Previsão de Vendas.  
    - Série Temporal e Regressão.  
    - O valor total das vendas ao final da 6ª semana.

# 0.0. Imports

In [124]:
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### 0.1. Loading datas

In [125]:
df_sales_raw = pd.read_csv("datas/train.csv", low_memory=False)
df_store_raw = pd.read_csv("datas/store.csv", low_memory=False)

# merge
df_raw = pd.merge(df_sales_raw,df_store_raw, on="Store", how="left")
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  object 
 11  CompetitionDistance        1014567 non-null  float64
 12  CompetitionOpenSinceMonth  693861 non-null   float64
 13  CompetitionO

In [126]:
df_raw[df_raw['Sales'] == 0]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
291,292,5,2015-07-31,0,0,0,1,0,1,a,a,1100.0,6.0,2009.0,0,NaN,NaN,NaN
875,876,5,2015-07-31,0,0,0,1,0,1,a,a,21790.0,4.0,2005.0,1,18.0,2015.0,"Feb,May,Aug,Nov"
1406,292,4,2015-07-30,0,0,0,1,0,1,a,a,1100.0,6.0,2009.0,0,NaN,NaN,NaN
1990,876,4,2015-07-30,0,0,0,1,0,1,a,a,21790.0,4.0,2005.0,1,18.0,2015.0,"Feb,May,Aug,Nov"
2521,292,3,2015-07-29,0,0,0,1,0,1,a,a,1100.0,6.0,2009.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1017205,1112,2,2013-01-01,0,0,0,0,a,1,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1017206,1113,2,2013-01-01,0,0,0,0,a,1,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1017207,1114,2,2013-01-01,0,0,0,0,a,1,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


# 1.0. Assessments

### 1.1. Data Dimensions

In [127]:
print(f"Linhas: {df_raw.shape[0]}")
print(f"Colunas: {df_raw.shape[1]}")

Linhas: 1017209
Colunas: 18


### 1.2. Data Types

Alterar a variável "Date" de Object para Datetime

In [128]:
df_raw["Date"] = pd.to_datetime(df_raw["Date"])

In [129]:
df_raw["Promo"] = df_raw["Promo"].astype('bool')
df_raw["SchoolHoliday"] = df_raw["SchoolHoliday"].astype('bool')

In [130]:
df_raw["StateHoliday"] = df_raw["StateHoliday"].astype('category')
df_raw["StoreType"] = df_raw["StoreType"].astype('category')
df_raw["Assortment"] = df_raw["Assortment"].astype('category')

### 1.3. Data Checkup NA

In [131]:
df_raw.isna().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance            2642
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
dtype: int64

### 1.4. Data Fillout

In [132]:
# CompetitionDistance
df_raw['CompetitionDistance'] = df_raw['CompetitionDistance'].apply(lambda x: 200000.0 if pd.isna(x) else x)

### 1.5. Drop Columns

Em variáveis com quantidade enorme de ausência de dados.

In [133]:
columns_drop = ['Open', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'Promo2']

In [134]:
df_raw.drop(columns=columns_drop, inplace=True)

In [135]:
df_raw.isna().sum()

Store                  0
DayOfWeek              0
Date                   0
Sales                  0
Customers              0
Promo                  0
StateHoliday           0
SchoolHoliday          0
StoreType              0
Assortment             0
CompetitionDistance    0
dtype: int64

In [136]:
df_raw = df_raw[df_raw["Sales"] != 0] 

### 1.6. Data Rename Columns

As variaveis estão em camelcase, vamos mudá-las para snakecase.

In [137]:
def convert_to_snake_case(df):
    return df.rename(columns=lambda x: x.lower().replace(' ', '_'))

In [138]:
df_raw = convert_to_snake_case(df_raw)

# 2.0. Descriptive Statistics

### 2.1. Variáveis Gerais

In [139]:
df_num = df_raw.select_dtypes(include=['int64', 'float64']).drop("store", axis=1)
df_obj = df_raw.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

In [140]:
def my_describe(df):
    d1 = pd.DataFrame(df.apply( np.mean )).T
    d2 = pd.DataFrame(df.apply( np.median )).T
    d3 = pd.DataFrame(df.apply( max )).T
    d4 = pd.DataFrame(df.apply( min )).T
    d5 = pd.DataFrame(df.apply( lambda x: x.max() - x.min() )).T
    d6 = pd.DataFrame(df.apply( np.std )).T
    d7 = pd.DataFrame(df.apply( lambda x: x.kurtosis())).T
    d8 = pd.DataFrame(df.apply( lambda x: x.skew() )).T
    d9 = pd.DataFrame(df.apply( lambda x: (x.std() /x.mean()) * 100  )).T
    result = pd.concat([d1,d2,d3,d4,d5,d6,d7,d8,d9]).T.reset_index()
    result.columns = ['index', 'mean', 'mediana', 'max', 'min', 'range', 'std', 'kurtosis', 'skew', 'Coef. Variação']
    return result.set_index("index")

In [141]:
describe = my_describe(df_num[['sales', 'customers']])
describe

,mean,mediana,max,min,range,std,kurtosis,skew,Coef. Variação
index,,,,,,,,,
sales,6955.959134,6369.0,41551.0,46.0,41505.0,3103.813677,4.854027,1.594929,44.620957
customers,762.777166,676.0,7388.0,8.0,7380.0,401.193915,13.318060,2.789062,52.596508


# 3.0. Feature Engineering

### 3.1. Hypothesis

**1.** As vendas aumentam de acordo com o crescimento de clientes?   
**2.** Lojas com maior sortimento tem suas vendas altas?  
**3.** Lojas com sortimentos maiores tendem a atrair mais clientes?  
**4.** As Lojas que tem seus competidores distantes vendem mais?  
**5.** O feriado do Natal impacta positivamente nas vende? O quanto?  
**6.** Com o passar dos anos, os feriado aumentam as vendas?  
**7.** Lojas do tipo 'A' impactam postivamente no aumento de número de clientes?   
**8.** Lojas com promoções tendem a vender mais?

### 3.2. Build Feature

In [142]:
# year
df_raw['year'] = df_raw['date'].dt.year

# month
df_raw['months'] = df_raw['date'].dt.month

# day
df_raw['day'] = df_raw['date'].dt.day

# dayofweek
dias_semana = {1: 'monday', 2:'tuesday', 3:'wednesday', 4:'thursday', 5:'friday', 6:'saturday', 7:'sunday'}
df_raw['daily-week'] = df_raw['dayofweek'].replace(dias_semana).astype("category")

# assortment
assortment_type = {'a': 'basic','b': 'extra', 'c': 'extended'}
df_raw['assortments'] = df_raw['assortment'].replace(assortment_type).astype('category')

#### 3.3.1. Drop columns

In [143]:
df_raw.drop(['dayofweek','date', 'assortment'], axis=1, inplace=True)

In [146]:
df_raw

,store,sales,customers,promo,stateholiday,schoolholiday,storetype,competitiondistance,year,months,day,daily-week,assortments
0,1,5263,555,True,0,True,c,1270.0,2015,7,31,friday,basic
1,2,6064,625,True,0,True,a,570.0,2015,7,31,friday,basic
2,3,8314,821,True,0,True,a,14130.0,2015,7,31,friday,basic
3,4,13995,1498,True,0,True,c,620.0,2015,7,31,friday,extended
4,5,4822,559,True,0,True,a,29910.0,2015,7,31,friday,basic
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016776,682,3375,566,False,a,True,b,150.0,2013,1,1,tuesday,basic
1016827,733,10765,2377,False,a,True,b,860.0,2013,1,1,tuesday,extra
1016863,769,5035,1248,False,a,True,b,840.0,2013,1,1,tuesday,extra
1017042,948,4491,1039,False,a,True,b,1430.0,2013,1,1,tuesday,extra


# 4.0. Export

In [148]:
df_raw.to_csv("datas/to_eda.csv", index=False)